In [1]:
import numpy as np
import pandas as pd
import itertools
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
df=pd.read_excel('C:\\Users\\dalemos\\Downloads\\FakeRecogna.xlsx', sheet_name = 'Sheet1')

In [3]:
df.shape

(11903, 8)

In [4]:
df.head()

,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,Boato – Ocorreu um apagão no Vaticano. O papa ...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,\nEdgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,NaN,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,27/03/2020 18h25,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,NaN,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,07/08/2020 13h42,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,NaN,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,27/04/2020 14h53,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,NaN,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11903 entries, 0 to 11902
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Titulo     11872 non-null  object 
 1   Subtitulo  5580 non-null   object 
 2   Noticia    11902 non-null  object 
 3   Categoria  11902 non-null  object 
 4   Data       11551 non-null  object 
 5   Autor      11886 non-null  object 
 6   URL        11902 non-null  object 
 7   Classe     11902 non-null  float64
dtypes: float64(1), object(7)
memory usage: 744.1+ KB


In [6]:
#validar % de nulls
df.isnull().sum() / df.shape[0] * 100

Titulo        0.260439
Subtitulo    53.121062
Noticia       0.008401
Categoria     0.008401
Data          2.957238
Autor         0.142821
URL           0.008401
Classe        0.008401
dtype: float64

In [7]:
#com a falta de 53% de dados na subcategoria, eliminamos esta coluna do dataset
df.drop('Subtitulo', inplace=True, axis=1)

In [8]:
df.describe()

,Classe
count,11902.000000
mean,0.500000
std,0.500021
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [9]:
df.Autor.value_counts()

\nEdgard Matsuki                                                   1498
\nKyene Becker                                                      599
 Por G1                                                             541
gilmar lopes                                                        478
Comprova                                                            388
                                                                   ... 
17/04/2020 14h11                                                      1
Rodrigo Vasconcelos, com informações da SapsMinistério da Saúde       1
25/06/2020 00h23                                                      1
15/04/2021 19h29                                                      1
28/08/2020 04h00                                                      1
Name: Autor, Length: 4405, dtype: int64

In [10]:
df.Categoria.value_counts()

saúde             4456
política          3951
entretenimento    1409
brasil             904
ciência            602
mundo              580
Name: Categoria, dtype: int64

In [11]:
#example of the data that have multiple values
df.Titulo.value_counts()

Veja o que é #FATO ou #FAKE nas declarações dos candidatos à Prefeitura de SP na 6ª semana de campanha                 12
Veja o que é #FATO ou #FAKE nas declarações dos candidatos à Prefeitura do Rio na 2ª semana de campanha do 2º turno    12
Veja o que é #FATO ou #FAKE nas falas dos candidatos à Prefeitura do Rio no debate da Globo                            10
Veja o que é #FATO ou #FAKE nas declarações dos candidatos à Prefeitura de SP na 4ª semana de campanha                  9
Veja o que é #FATO ou #FAKE nas declarações do ex-ministro Pazuello na CPI da Covid                                     9
                                                                                                                       ..
mosaico mostrar o profissional saudar morrer pelar covid-19 o brasil                                                    1
Acre recebe mais 13,3 mil doses da vacina da AstraZeneca/Oxford                                                         1
foto mostrar o bancada p

In [12]:
#example of the data that have multiple values
df[df.Titulo == "Veja o que é #FATO ou #FAKE nas declarações dos candidatos à Prefeitura de SP na 6ª semana de campanha"]

,Titulo,Noticia,Categoria,Data,Autor,URL,Classe
11849,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11850,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11851,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11852,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11853,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11854,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11855,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11856,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11857,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0
11858,Veja o que é #FATO ou #FAKE nas declarações do...,candidato prefeitura paulo continuidade campan...,política,07/11/2020,"Por G1, O Globo, Extra, TV Globo, CBN, Valor ...",https://g1.globo.com/fato-ou-fake/noticia/2020...,0.0


In [13]:
#drop duplicates values and keep the first records from duplicates
#with an assumption that the first row contains the latest data 

df = df.drop_duplicates("Titulo", keep='first')
df= df.reset_index(drop=True)

In [14]:
df.head()

,Titulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,\nEdgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,27/03/2020 18h25,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,07/08/2020 13h42,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,27/04/2020 14h53,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


In [15]:
#remover da coluna "Autor" os casos com data
df["Autor"].fillna("none", inplace= True)
df.loc[df.Autor.str.contains("20"), "Autor"] = 'none'
df["Autor"] = df["Autor"].str.replace("\n", "")

In [16]:
df.head()

,Titulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,Edgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,none,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,none,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,none,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


In [17]:
#obter o nome do URL base
for i in range(df.shape[0]):
    url = df.iloc[i,5]
    dominio = urlparse(url).netloc
    print (dominio)
    df.loc[i, "Dominio"] = dominio
    


www.boatos.org
noticias.uol.com.br
www.uol.com.br
economia.uol.com.br
www.e-farsas.com
www.boatos.org
noticias.uol.com.br
www.boatos.org
g1.globo.com
www.boatos.org
www.gov.br
www.boatos.org
extra.globo.com
www.uol.com.br
www.gov.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
g1.globo.com
projetocomprova.com.br
checamos.afp.com
www.boatos.org
www.e-farsas.com
www.boatos.org
www.boatos.org
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
projetocomprova.com.br
www.uol.com.br
noticias.uol.com.br
projetocomprova.com.br
tvefamosos.uol.com.br
www.boatos.org
noticias.uol.com.br
www.uol.com.br
noticias.uol.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.uol.com.br
g1.globo.com
noticias.uol.com.br
www.boatos.org
projetocomprova.com.br
noticias.uol.com.br
g1.globo.com
g1.globo.com
www.e-farsas.com
noticias.uol.com.

www.e-farsas.com
www.boatos.org
www.boatos.org
noticias.uol.com.br
www.boatos.org
www.gov.br
www.boatos.org
noticias.uol.com.br
g1.globo.com
www.e-farsas.com
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
www.uol.com.br
economia.uol.com.br
www.uol.com.br
noticias.uol.com.br
tvefamosos.uol.com.br
noticias.uol.com.br
g1.globo.com
economia.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
projetocomprova.com.br
g1.globo.com
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
checamos.afp.com
noticias.uol.com.br
economia.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
www.boatos.org
www.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
extra.globo.com
projetocomprova.com.br
www.boatos.org
g1.globo.com
extra.globo.com
www.boatos.org
g1.globo.com
g1.globo.com
noticias.uol.com.br


g1.globo.com
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
www.boatos.org
www.boatos.org
www.e-farsas.com
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
www.boatos.org
www.boatos.org
g1.globo.com
noticias.uol.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
projetocomprova.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
educacao.uol.com.br
www.boatos.org
economia.uol.com.br
g1.globo.com
g1.globo.com
checamos.afp.com
g1.globo.com
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
noticias.uol.com.br
www.e-farsas.com
g1.globo.com
economia.uol.com.br
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
g1.globo.com
www.uol.com.br
www.boatos.org
noticias.uol.com.br
entretenimento.uol.com.br
noticias.uol.com.br
www.boatos.org


www.boatos.org
checamos.afp.com
www.boatos.org
www.boatos.org
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
www.boatos.org
g1.globo.com
noticias.uol.com.br
g1.globo.com
www.boatos.org
g1.globo.com
g1.globo.com
noticias.uol.com.br
g1.globo.com
g1.globo.com
g1.globo.com
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
www.e-farsas.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
projetocomprova.com.br
www.e-farsas.com
g1.globo.com
projetocomprova.com.br
noticias.uol.com.br
g1.globo.com
www.boatos.org
www.uol.com.br
www.boatos.org
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
www.boatos.org
economia.uol.com.br
www.boatos.org
g1.globo.com
www.boatos.org
www.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
www.e-farsas.com
educacao.uol.com.br
www.boatos.org
noticias.uol.com.br
notic

noticias.uol.com.br
g1.globo.com
www.uol.com.br
www.e-farsas.com
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
checamos.afp.com
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
economia.uol.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
www.boatos.org
g1.globo.com
www.gov.br
projetocomprova.com.br
entretenimento.uol.com.br
extra.globo.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
noticias.uol.com.br
www.uol.com.br
www.boatos.org
www.e-farsas.com
tvefamosos.uol.com.br
www.boatos.org
noticias.uol.com.br
g1.globo.com
www.e-farsas.com
noticias.uol.com.br
projetocomprova.com.br
noticias.uol.com.br
economia.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
noticias.uol.com.br
g1.globo.com
checamos.afp.com
noticias.uol.com.br
noticias.uol.com.br
projetocomprova.com.br
g1.globo.com
g1.globo.com
www.boatos.org
economia.uol.com.br
g1.globo.com
economia.uol.com.br
g1.globo.com
g1.globo.com
noticias.uol.com.br
noticias.uol.co

...

noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.e-farsas.com
g1.globo.com
noticias.uol.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
g1.globo.com
g1.globo.com
g1.globo.com
www.boatos.org
extra.globo.com
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.gov.br
noticias.uol.com.br
g1.globo.com
tvefamosos.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
www.e-farsas.com
checamos.afp.com
g1.globo.com
noticias.uol.com.br
www.uol.com.br
economia.uol.com.br
www.uol.com.br
www.e-farsas.com
www.e-farsas.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.

noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
www.gov.br
www.boatos.org
www.boatos.org
projetocomprova.com.br
noticias.uol.com.br
g1.globo.com
www.boatos.org
noticias.uol.com.br
projetocomprova.com.br
checamos.afp.com
www.e-farsas.com
extra.globo.com
www.boatos.org
www.e-farsas.com
noticias.uol.com.br
www.e-farsas.com
economia.uol.com.br
g1.globo.com
checamos.afp.com
noticias.uol.com.br
www.uol.com.br
g1.globo.com
www.e-farsas.com
g1.globo.com
www.boatos.org
noticias.uol.com.br
projetocomprova.com.br
www.e-farsas.com
www.boatos.org
economia.uol.com.br
g1.globo.com
noticias.uol.com.br
www.boatos.org
checamos.afp.com
www.boatos.org
www.boatos.org
g1.globo.com
noticias.uol.com.br
g1.globo.com
g1.globo.com
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.e-farsas.com
www.gov.br
checamos.afp.com
www.uol.com.br
www.boatos.org
www.boatos.org
www.boatos.org
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
extra.globo.com
noticias.uol.com.br
www.e-farsas.co

g1.globo.com
noticias.uol.com.br
www.uol.com.br
www.boatos.org
www.boatos.org
noticias.uol.com.br
economia.uol.com.br
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
checamos.afp.com
projetocomprova.com.br
g1.globo.com
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
noticias.uol.com.br
extra.globo.com
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
www.boatos.org
www.boatos.org
noticias.uol.com.br
www.uol.com.br
noticias.uol.com.br
economia.uol.com.br
projetocomprova.com.br
noticias.uol.com.br
www.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
www.boatos.org
www.gov.br
www.boatos.org
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
www.boatos.org
www.boatos.org
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
checamos.afp.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
www.boatos.org
noticias.uol.co

entretenimento.uol.com.br
noticias.uol.com.br
www.boatos.org
g1.globo.com
g1.globo.com
extra.globo.com
g1.globo.com
www.boatos.org
g1.globo.com
www.boatos.org
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.uol.com.br
g1.globo.com
g1.globo.com
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
entretenimento.uol.com.br
noticias.uol.com.br
www.e-farsas.com
noticias.uol.com.br
g1.globo.com
www.boatos.org
www.uol.com.br
www.gov.br
noticias.uol.com.br
www.boatos.org
www.uol.com.br
noticias.uol.com.br
www.boatos.org
g1.globo.com
projetocomprova.com.br
g1.globo.com
checamos.afp.com
noticias.uol.com.br
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
www.e-farsas.com
noticias.uol.com.br
www.boatos.org
www.boatos.org
extra.globo.com
g1.globo.com
noticias.uol.com.br
www.e-farsas.com
www.boatos.org
extra.globo.com
www.boatos.org
noticias.uol.com.br
www.boatos.org
www.e-far

www.uol.com.br
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
www.uol.com.br
www.boatos.org
noticias.uol.com.br
www.boatos.org
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
www.boatos.org
extra.globo.com
checamos.afp.com
g1.globo.com
g1.globo.com
g1.globo.com
www.boatos.org
www.uol.com.br
g1.globo.com
noticias.uol.com.br
extra.globo.com
www.boatos.org
noticias.uol.com.br
www.boatos.org
www.boatos.org
g1.globo.com
entretenimento.uol.com.br
www.uol.com.br
noticias.uol.com.br
checamos.afp.com
noticias.uol.com.br
www.boatos.org
noticias.uol.com.br
noticias.uol.com.br
g1.globo.com
g1.globo.com
www.boatos.org
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
checamos.afp.com
g1.globo.com
www.e-farsas.com
noticias.uol.com.br
www.boatos.org
www.e-farsas.com
noticias.uol.com.br
www.boatos.org
www.e-farsas.com
g1.globo.com
noticias.uol.com.br
noticias.uol.com.br
www.gov.br
entretenimento.uol.com.br
noticias.uol.com.br
noticias.uol.com.br
extra.globo.com
www.boatos.org
www.uol

In [18]:
array_fake = df[df.Classe == 0.0].Dominio.unique()
array_real = df[df.Classe == 1.0].Dominio.unique()

In [19]:
s_real = set(array_real)-set(array_fake)
df_real = pd.DataFrame({'dominio': data} for data in s_real)
#adicionar à lista mais alguns sites confiáveis
data = ['www.folha.uol.com.br', 'www.metropoles.com', 'www.estadao.com.br','www.cnnbrasil.com.br',
        'www.bbc.com/portuguese','exame.com','oglobo.globo.com','www.globo.com'] 
df2 = pd.DataFrame(data, columns = ['dominio'])
df_real.append(df2)



,dominio
0,tab.uol.com.br
1,entretenimento.uol.com.br
2,www.gov.br
3,extra.globo.com
4,tvefamosos.uol.com.br
5,educacao.uol.com.br
6,www.uol.com.br
7,economia.uol.com.br
0,www.folha.uol.com.br
1,www.metropoles.com


In [20]:
array_fake = df[df.Classe == 0.0].Autor.unique()
array_real = df[df.Classe == 1.0].Autor.unique()

In [21]:
s_real = set(array_real)-set(array_fake)
df_autor_real = pd.DataFrame({'autor': data} for data in s_real)
df_autor_real

,autor
0,"Por RFI, G1"
1,Kelli Kadanus
2,Renata Cafardo
3,Rafael Moraes Moura e Jussara Soares
4,Naomi Tajitsu
...,...
1218,"Por Vanessa Silvestre e Márcio Falcão, GloboN..."
1219,Nathan Lopes*
1220,Do G1
1221,Renata Mariz


In [22]:
'''
outros sites
Confiável: 
www.folha.uol.com.br
www.metropoles.com
www.estadao.com.br
www.cnnbrasil.com.br
www.bbc.com/portuguese
exame.com
oglobo.globo.com
www.globo.com
'''


'\noutros sites\nConfiável: \nwww.folha.uol.com.br\nwww.metropoles.com\nwww.estadao.com.br\nwww.cnnbrasil.com.br\nwww.bbc.com/portuguese\nexame.com\noglobo.globo.com\nwww.globo.com\n'

# treinar o modelo


In [23]:
#FakeRecogna - Get the labels
target=df.Classe
target.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Classe, dtype: float64

In [24]:
#FakeRecogna - Split the dataset
x_train,x_test,y_train,y_test=train_test_split(df['Noticia'], target, test_size=0.2, random_state=7)

In [25]:
#TF-IDF é uma abreviação de Term Frequency Inverse Document Frequency
#https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275
tfidf_vectorizer=TfidfVectorizer(max_df=0.7)

In [26]:
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [27]:
#https://medium.com/geekculture/passive-aggressive-algorithm-for-big-data-models-8cd535ceb2e6
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [28]:
type(target)

pandas.core.series.Series

### ver com os dados reservados para teste o quão bom é o modelo para acertar

In [29]:
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.44%


In [30]:
confusion_matrix(y_test,y_pred, labels=[0,1])

array([[1078,   79],
       [  74, 1102]], dtype=int64)

In [31]:
print("Classification report:\n\n{}".format(classification_report(y_test, y_pred)))

Classification report:

              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      1157
         1.0       0.93      0.94      0.94      1176

    accuracy                           0.93      2333
   macro avg       0.93      0.93      0.93      2333
weighted avg       0.93      0.93      0.93      2333



In [195]:
y_pred

array([1., 1., 1., ..., 1., 0., 0.])

In [32]:
#https://www.boatos.org/politica/lula-e-recebido-com-gritos-de-lula-ladrao-no-aeroporto-de-campinas-viracopos.html
#https://www.boatos.org/politica/onu-desmentiu-comite-inocentou-ex-presidente-lula.html
#https://www.boatos.org/politica/lula-e-envergonhado-por-repentista-em-comicio-no-nordeste.html
#https://www.boatos.org/politica/alexandre-de-moraes-video-ameacas-stf-quando-advogado.html
#https://www.boatos.org/politica/tse-proibe-bolsonaro-baixar-preco-dos-combustiveis.html
input_fake = pd.Series(['Vídeo mostra manifestantes gritando “Lula ladrão seu lugar é na prisão” durante chegada do ex-presidente Lula ao aeroporto de Viracopos, em Campinas, em maio de 2022', 
                        'A ONU veio a público desmentir a informação que aponta que um comitê inocentou o ex-presidente das acusações de corrupção.',
                        'Em palco de evento, repentista nordestino envergonha Lula sobre obra de transposição do São Francisco e elogio Bolsonaro',
                        'Quando era advogado, Alexandre de Moraes fez um vídeo com ameaças ao STF. Ele disse que iria tocar fogo no Supremo Tribunal Federal e atirar em ministros.',
                        'TSE proíbe Bolsonaro de reduzir o preço dos combustíveis'])


tfidf_fake=tfidf_vectorizer.transform(input_fake)
pac.predict(tfidf_fake)

array([0., 1., 0., 0., 0.])

In [33]:
#https://www.folha.uol.com.br/poder/2022/05/bolsonaro-tenta-barrar-relatorio-sobre-assessor-apos-ordem-de-moraes-a-pf.shtml
#https://noticias.uol.com.br/politica/ultimas-noticias/2022/05/06/ana-cristina-valle-ex-mulher-bolsonaro-chantagem-ex-funcionario.htm
#https://noticias.uol.com.br/politica/ultimas-noticias/2022/05/06/gasto-r-42-milhoes-bolsonaro-cartao-corporativo-presidencia-35-dias.htm
#https://noticias.uol.com.br/politica/ultimas-noticias/2022/05/06/autor-da-facada-em-bolsonaro-adelio-pode-ser-solto-apos-nova-pericia.htm
#https://noticias.uol.com.br/politica/ultimas-noticias/2022/05/07/o-que-lula-e-bolsonaro-ja-falaram-sobre-a-guerra-na-ucrania.htm
#https://www.metropoles.com/brasil/eleicoes-2022/bolsonaro-lanca-ofensiva-para-reduzir-rejeicao-entre-mulheres-e-jovens
input_real = pd.Series(['A Advocacia-Geral da União contestou, nesta sexta-feira (6), a decisão do ministro Alexandre de Moraes, do STF (Supremo Tribunal Federal), de mandar a Poícia Federal produzir um relatório sobre o coronel Mauro Cid, ajudante de ordens do presidente Jair Bolsonaro (PL).',
                        'Marcelo Nogueira dos Santos, que trabalhou como caseiro para a advogada Ana Cristina Valle, ex-mulher do presidente Jair Bolsonaro (PL), enviou uma série de mensagens com supostas chantagens à ex-patroa, ameaçando revelar o que sabe sobre um suposto esquema das rachadinhas envolvendo a família. As informações são da revista Veja.',
                        'O presidente Jair Bolsonaro (PL) gastou cerca de R$ 4,2 milhões nos cartões corporativos da Presidência da República nos últimos 35 dias. O levantamento foi feito pelo deputado federal Elias Vaz (PSB-GO) por meio do Portal da Transparência',
                        'Adélio Bispo, autor da facada no presidente Jair Bolsonaro (PL) em 2018, deve passar por nova perícia médica no próximo mês para avaliar se ainda representa perigo à sociedade e pode ser solto. A solicitação de nova análise foi feita pelo MPF (Ministério Público Federal) nesta semana',
                        'Deram o que falar as recentes declarações do ex-presidente Luiz Inácio Lula da Silva (PT), em entrevista à revista Time, sobre culpa do presidente da Ucrânia, Volodymyr Zelensky, na guerra da Ucrânia. "Esse cara é tão responsável quanto o Putin, porque numa guerra não tem apenas um culpado", disse Lula.',
                        'A pré-campanha à reeleição do presidente Jair Bolsonaro (PL) já está colocando em prática ações para tentar reduzir a rejeição do atual mandatário do país entre dois setores relevantes do eleitorado: as mulheres e os jovens. Os articuladores políticos do titular chefe do Executivo federal avaliam que ele precisa ganhar votos desses públicos para ameaçar a vantagem do oponente Luiz Inácio Lula da Silva (PT) nas pesquisas e se tornar mais competitivo.'])


tfidf_real=tfidf_vectorizer.transform(input_real)
a = pac.predict(tfidf_real)

In [34]:
a

array([1., 1., 1., 1., 1., 1.])

### Tentativa de explicar como funciona o modelo em estrutura vetorial

In [194]:
text = ["The cycle is ridden on the track.",
    "The bus is driven on the road.",
    "He is driving the bus."]

# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
vectorizer.fit_transform(text)

# summarize
print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names())
print(vectorizer.idf_)

{'the': 9, 'cycle': 1, 'is': 5, 'ridden': 7, 'on': 6, 'track': 10, 'bus': 0, 'driven': 2, 'road': 8, 'he': 4, 'driving': 3}
['bus', 'cycle', 'driven', 'driving', 'he', 'is', 'on', 'ridden', 'road', 'the', 'track']
[1.28768207 1.69314718 1.69314718 1.69314718 1.69314718 1.
 1.28768207 1.69314718 1.69314718 1.         1.69314718]


In [ ]:
['bus',     'cycle',   'driven',  'driving', 'he',     'is', 'on',      'ridden',  'road',    'the',     'track']
[1.28768207 1.69314718 1.69314718 1.69314718 1.69314718 1.   1.28768207 1.69314718 1.69314718 1.         1.69314718]

In [333]:
a = 'Guilherme Caetano'
d = pd.DataFrame([2, 3], columns=['x'])
# in df_autor_real.autor
a in df_autor_real.values

True